<a target="_blank" href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2024/blob/main/Lab01/Assignment1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Assignment 1 (10 points)**

## **Solving a linear system in python**

In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.

We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:
$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**
- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms

### **1. Parsing the System of Equations (1 point)**

The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**
```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated

In [13]:
import pathlib

def parse_equation(equation: str) -> tuple[list[float], float]:
    left_side, right_side = equation.split('=')
    left_side = left_side.replace(' ', '')
    
    coeffs = [0, 0, 0]
    variables = ['x', 'y', 'z']
    
    current_coeff = ''
    sign = 1  # by default positive
    
    for char in left_side:
        if char in ['+', '-']:
            if current_coeff:
                coeffs[variables.index(variable)] = sign * (int(current_coeff) if current_coeff else 1)
            current_coeff = ''
            sign = 1 if char == '+' else -1
        elif char in variables:
            variable = char
            coeffs[variables.index(variable)] = sign * (int(current_coeff) if current_coeff else 1)
            current_coeff = ''
        else:
            current_coeff += char

    result = int(right_side.strip())
    
    return coeffs, result

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    A = []
    B = []

    with path.open() as f:
        for line in f:
            coeffs, result = parse_equation(line.strip())
            A.append(coeffs)
            B.append(result)

    return A, B

A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")

A=[[2, 3, -1], [1, -1, 4], [3, 1, 2]] B=[5, 6, 7]


### **2. Matrix and Vector Operations (5 points)**

Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:

#### 2.1. Determinant

Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:
$$ \text{det}(A) = a_{11}(a_{22}a_{33} - a_{23}a_{32}) - a_{12}(a_{21}a_{33} - a_{23}a_{31}) + a_{13}(a_{21}a_{32} - a_{22}a_{31}) $$

In [24]:
print(A)
def determinant(matrix: list[list[float]]) -> float:
    if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    
    a11, a12, a13 = matrix[0]
    a21, a22, a23 = matrix[1]
    a31, a32, a33 = matrix[2]
    
    det = (a11 * (a22 * a33 - a23 * a32)
          - a12 * (a21 * a33 - a23 * a31)
          + a13 * (a21 * a32 - a22 * a31))
    
    return det

print(f"{determinant(A)=}")

[[2, 3, -1], [1, -1, 4], [3, 1, 2]]
determinant(A)=14


#### 2.2. Trace

Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a_{11} + a_{22} + a_{33} $$

In [17]:
def trace(matrix: list[list[float]]) -> float:
    a11, a12, a13 = matrix[0]
    a21, a22, a23 = matrix[1]
    a31, a32, a33 = matrix[2]

    trace = a11 + a22 + a33

    return trace

print(f"{trace(A)=}")

trace(A)=3


#### 2.3. Vector norm

Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$

In [18]:
def norm(vector: list[float]) -> float:
    sum_of_squares = sum(v ** 2 for v in vector)

    return sum_of_squares ** 0.5

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.
    

In [20]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
    rows = len(matrix)
    cols = len(matrix[0])

    transposed_matrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            transposed_matrix[j][i] = matrix[i][j]
    
    return transposed_matrix

print(f"{transpose(A)=}")

transpose(A)=[[2, 1, 3], [3, -1, 1], [-1, 4, 2]]


#### 2.5. Matrix-vector multiplication

Write a function that multiplies matrix $A$ with vector $B$.

In [21]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    result = [0 for _ in range(len(matrix))]

    for i in range(len(matrix)):
        for j in range(len(vector)):
            result[i] += matrix[i][j] * vector[j]
    
    return result

print(f"{multiply(A, B)=}")

multiply(A, B)=[21, 27, 35]


### **3. Solving using Cramer's Rule (1 point)**

Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.

In [22]:
def replace_column(matrix: list[list[float]], vector: list[float], col_idx: int) -> list[list[float]]:
    new_matrix = [row[:] for row in matrix]
    for i in range(len(matrix)):
        new_matrix[i][col_idx] = vector[i]
    return new_matrix

def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    det_A = determinant(matrix)
    
    if det_A == 0:
        raise ValueError("The system don't have a single solution because the determinant is 0.")
    
    solutions = []
    for i in range(len(vector)):
        modified_matrix = replace_column(matrix, vector, i)
        det_Ai = determinant(modified_matrix)
        solutions.append(det_Ai / det_A)
    
    return solutions

print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**

Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M_{ij}) $$
where $M_{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.

In [32]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    return [[matrix[x][y] for y in range(len(matrix)) if y != j] 
            for x in range(len(matrix)) if x != i]

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    cofactors = []
    for i in range(len(matrix)):
        row = []
        for j in range(len(matrix)):
            sign = (-1) ** (i + j)
            minor_matrix = minor(matrix, i, j)
            row.append(sign * determinant(minor_matrix))
        cofactors.append(row)
    return cofactors

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    return transpose(cofactor(matrix))

def inverse(matrix: list[list[float]]) -> list[list[float]]:
    det_A = determinant(matrix)
    
    if det_A == 0:
        raise ValueError("The matrix don't have inverse because the determinant is 0.")
    
    adj = adjoint(matrix)
    
    return [[adj[i][j] / det_A for j in range(len(matrix))] for i in range(len(matrix))]

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    inv_A = inverse(matrix)
    return multiply(inv_A, vector)

print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285765, 2.071428571428571, 1.9285714285714288]
